In [1]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket
from etrade.spot.forecast.plan_a.constructor import *
from etrade.spot.forecast.yieldindex import zero_quantile
from etrade.spot.forecast import yieldindex
from  data_utils.stochastic_utils.vdistributions.abstract import AbstractDistribution
from  data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, SkewNormalDistribution
from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.kernel2 import KernelMixDistribution
from data_utils.stochastic_utils.vdistributions.tools.divergence import kl_divergence_continuous, crps, quantile_RMSE

from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.mfk.skewnd2 import dof_snd_fitter

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [2]:
aq = OrdinaryGaussianKernelDistributionConstructor((0, 50), (0.1, 10), (2, 8))
dp = OrdinaryGaussianKernelDistributionConstructor((0, 10), (0.1, 10), (2, 8))
rp = OrdinaryGaussianKernelDistributionConstructor((0, 10), (0.1, 10), (2, 8))

In [3]:
mc = MarketConstructor(aq, dp, rp)
s = Station("station", 50)
br = PointwiseRecycle(0.5,1.05)

In [4]:
class MixedMarket:
    def __init__(self, n0, n1):
        self.n0 = n0
        self.n1 = n1
        self.real_market = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))
        self.noice_market = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))
        self.predicted_market = market_hybridization(self.real_market, self.noice_market, self.n0, self.n1, (0, 50), (0, numpy.inf), (0, numpy.inf))
        
    def refresh(self):
        self.real_market = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))
        self.noice_market = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))
        self.predicted_market = market_hybridization(self.real_market, self.noice_market, self.n0, self.n1, (0, 50), (0, numpy.inf), (0, numpy.inf))
        
    
    def observed(self):
        return self.real_market.observe()
    
    def optimize(self, station:Station, recycle: BasicRecycle, rounds=1000):
        return self.predicted_market.power_generation_optimizer(station, recycle, num=rounds).x
    
    def faster_optimize(self, station:Station, recycle: BasicRecycle, rounds=1000):
        return self.predicted_market.faster_power_generation_optimizer(station, recycle, num=rounds).x
    
    def zero_quantile(self, station:Station, recycle: BasicRecycle, rounds=1000, faster=False):
        if faster:
            x = self.faster_optimize(station, recycle, rounds)
        else:
            x = self.optimize(station, recycle, rounds)
        return zero_quantile(
            self.real_market.market_trade(station, recycle, x, num=rounds),
            self.real_market.market_trade(station, recycle, self.real_market.mean(rounds)[0], num=rounds)
        )

In [6]:
l = []
for _ in range(20):
    mm = MixedMarket(1000, 500)
    l.append(mm.zero_quantile(s, br, 500, faster=False))
    # l.append(mm.faster_optimize(s, br, 1000))
    # mm.predicted_market.rvf(100)
    # mm.refresh()
    # print(mm.real_market)
print(l)

E:\code\github\data_utils\data_utils\stochastic_utils\vdistributions\parameter\continuous\basic.py:63: RuntimeWarning: divide by zero encountered in divide
  return 0.5 * erfc((self.mu - x) / (numpy.sqrt(2) * self.sigma))
E:\code\github\data_utils\data_utils\stochastic_utils\vdistributions\parameter\continuous\basic.py:63: RuntimeWarning: invalid value encountered in divide
  return 0.5 * erfc((self.mu - x) / (numpy.sqrt(2) * self.sigma))
E:\code\github\data_utils\data_utils\stochastic_utils\vdistributions\parameter\continuous\basic.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  return 0.5 * erfc((self.mu - x) / (numpy.sqrt(2) * self.sigma))


[0.502, 0.412, 0.414, 0.25, 0.314, 0.932, 0.328, 0.39, 0.284, 0.454, 0.646, 0.25, 0.344, 0.392, 0.28, 0.478, 0.42, 0.4, 0.336, 0.652]
